In [257]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/kostin_001/anaconda3/envs/thesis/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'contour']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [258]:
import cv2
import numpy as np

In [259]:
import xml.etree.ElementTree as ET

In [260]:
from scipy.spatial import distance

In [261]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [262]:
from keras.models import model_from_json

In [263]:
import json

In [264]:
model = Sequential()

with open('model.json', 'r') as f:
    model = model_from_json(f.read())
    
model.load_weights('first_try.h5')

In [265]:
def preprocess_coords(xs, ys):
    distances = []
    for i in range(4):
        distances.append(distance.euclidean(np.float32([xs[i], ys[i]]), np.float32([xs[(i+1)%4], ys[(i+1)%4]])))
    starting_point=np.argmax(np.array(distances))
    new_xs = xs[starting_point:]+xs[:starting_point]
    new_ys = ys[starting_point:]+ys[:starting_point]
    return new_xs, new_ys

In [266]:
index = '1'

In [267]:
tree = ET.parse('%s.xml' % (index))
root = tree.getroot()

In [268]:
image = cv2.imread('%s.jpg' % (index), 0)
pts2 = np.float32([[0,60],[0,0],[40,0],[40,60]])
slots = []
ids = []
coord=[]
predictions = []
nop = np.array([None])

In [269]:
for space in root.findall('space'):
    space_id = space.get('id')
    ids.append(space_id)
    contour = space.find('contour')
    xs = []
    ys = []
    for point in contour.findall('point'):
        xs.append(point.get('x'))
        ys.append(point.get('y'))
    assert len(xs)==4
    assert len(ys)==4
    xs, ys = preprocess_coords(xs, ys)
    xs = np.float32(xs)
    ys = np.float32(ys)
    coords=np.vstack((xs, ys)).T
    coord.append(coords)
    M = cv2.getPerspectiveTransform(coords,pts2)
    dst = cv2.warpPerspective(image,M,(40,60))
    #cv2.imwrite("6_%s.jpg" % space_id, dst)
    slots.append(np.reshape(dst, (40, 60, 1)))
    #predictions.append(model.predict(dst, 16, 1))

In [270]:
pred = model.predict(np.array(slots), 16, 1)

100/100 [==============================] - 0s 974us/step


In [271]:
pred.ravel().shape, len(ids)

((100,), 100)

In [272]:
answer = {}
pred = pred.ravel().tolist()
for i in range(len(ids)):
    answer[ids[i]]='Occupied' if pred[i] else 'Empty'
    pts = np.array(coord[i], np.int32).reshape((-1, 1, 2))
    color = (0, 255, 0) if pred[i] else (255, 0, 0)
    cv2.polylines(image,[pts],True,color)
    
with open('%s_prediction.json' % (index), 'w') as f:
    f.write(json.dumps(answer))
    
cv2.imwrite('%s_marking.jpg'%(index), image)
answer

{'1': 'Empty',
 '10': 'Empty',
 '100': 'Occupied',
 '11': 'Occupied',
 '12': 'Empty',
 '13': 'Empty',
 '14': 'Empty',
 '15': 'Occupied',
 '16': 'Empty',
 '17': 'Empty',
 '18': 'Empty',
 '19': 'Empty',
 '2': 'Occupied',
 '20': 'Occupied',
 '21': 'Occupied',
 '22': 'Empty',
 '23': 'Empty',
 '24': 'Empty',
 '25': 'Empty',
 '26': 'Empty',
 '27': 'Empty',
 '28': 'Occupied',
 '29': 'Empty',
 '3': 'Empty',
 '30': 'Occupied',
 '31': 'Empty',
 '32': 'Occupied',
 '33': 'Occupied',
 '34': 'Occupied',
 '35': 'Occupied',
 '36': 'Occupied',
 '37': 'Empty',
 '38': 'Empty',
 '39': 'Empty',
 '4': 'Empty',
 '40': 'Empty',
 '41': 'Empty',
 '42': 'Occupied',
 '43': 'Empty',
 '44': 'Occupied',
 '45': 'Occupied',
 '46': 'Empty',
 '47': 'Empty',
 '48': 'Empty',
 '49': 'Empty',
 '5': 'Empty',
 '50': 'Empty',
 '51': 'Empty',
 '52': 'Occupied',
 '53': 'Empty',
 '54': 'Empty',
 '55': 'Empty',
 '56': 'Empty',
 '57': 'Occupied',
 '58': 'Empty',
 '59': 'Occupied',
 '6': 'Empty',
 '60': 'Occupied',
 '61': 'Empty',
 